## read csv

In [116]:
import pandas as pd
path = '~/CODE/csvprocessing/field_of_study_exercise.csv'
import re
from spellchecker import SpellChecker

In [65]:
data = pd.read_csv(path)
data.head(10)

,level,level_name,field_of_study,academic_field
0,NaN,NaN,"-, Marketing/Marketing Management, General",marketing
1,NaN,NaN,“Wellness Counseling Certificate”,NaN
2,NaN,NaN,"(BA) Hons Fashion, Fashion/Apparel Design",NaN
3,NaN,NaN,"(BA) Liberal Arts, Liberal Arts and Sciences/L...",NaN
4,NaN,NaN,* Career certificate of Marketing.,NaN
5,NaN,NaN,"1st Class Honour, Information System and Compu...",NaN
6,NaN,NaN,"2:1, Business Management",business management
7,NaN,NaN,"2:1, French and Hispanic Studies",NaN
8,NaN,NaN,"2.1, Graphic Design and Illustration",NaN
9,NaN,NaN,"2.1, Management",NaN


## Assess Missing Values:

In [66]:
data.info()
print("null count:")
null_counts = data.isnull()
print(null_counts.sum())
print("null ratios:")
print(null_counts.mean())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   level           322 non-null    object
 1   level_name      312 non-null    object
 2   field_of_study  375 non-null    object
 3   academic_field  141 non-null    object
dtypes: object(4)
memory usage: 11.8+ KB
null count:
level              53
level_name         63
field_of_study      0
academic_field    234
dtype: int64
null ratios:
level             0.141333
level_name        0.168000
field_of_study    0.000000
academic_field    0.624000
dtype: float64


Looking into the data, it easily figures out the  missing values of columns "level" should be "diploma" it is because 

In [67]:
# Clean column names
data.columns = data.columns.str.strip()
# Display cleaned column names
print(data.columns)

Index(['level', 'level_name', 'field_of_study', 'academic_field'], dtype='object')


In [74]:
# Define a regular expression to remove special characters
pattern = r'[^\w\s]'  # Matches characters that are not alphanumeric or whitespace
# Create an empty dictionary to store column-wise special characters
special_chars = set()
# Iterate through each column in the DataFrame
for col in data.columns:
    # Create an empty set for the current column's special characters
    

    # Iterate through each element in the column
    for element in data[col].values:
        # Find all special characters using the regular expression
        matches = re.findall(pattern, str(element))
        # Add unique characters to the set
        special_chars.update(matches)

# Print the dictionary showing special characters by column
print("Special characters:")
print(f"{sorted(special_chars)}")

Special characters:
['"', '#', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', ']', '’', '“', '”']


In [113]:
# Define the whitelist characters
# whitelist = r'\-\_\.'  # Include any characters you want to whitelist
whitelist =''
# Construct the regex pattern
pattern = f'[^a-zA-Z0-9\s{whitelist}]'
mapping_specific_pattern ={
    'bba':'',
    'honours': 'Hons',
    'bachelor degree in':'',
    'bachelor degree':'',
    'bachelor in':'',
    'bachelor of':'',
    'basc': '',
    '&': 'and',
    'bachelor hons in': ''
}

# Function to replace substrings regardless of case sensitivity
def replace_substrings(string, replacements):
    pattern = re.compile('|'.join(re.escape(key) for key in replacements.keys()), re.IGNORECASE)
    return pattern.sub(lambda match: replacements[match.group(0).lower()], string)

# Function to clean each field_of_study entry
def clean_field_of_study(field):
    # Split the field by ','
    fields = field.split(',')
    # replace special cases:
    cleaned_fields = [replace_substrings(s, mapping_specific_pattern) for s in fields]
    # Clean each split value
    cleaned_fields = [re.sub(pattern, '', f).strip() for f in cleaned_fields]
    # mapping specific patterns:
    cleaned_fields = [replace_substrings(s, mapping_specific_pattern) for s in cleaned_fields]
    # Remove empty fields
    cleaned_fields = [f for f in cleaned_fields if f]
    # Join them back with ','
    cleaned_text =  ', '.join(cleaned_fields)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  # Remove multiple spaces
    return cleaned_text
split_df = data['field_of_study'].apply(clean_field_of_study)

split_df.head(10)
split_df.to_csv('~/CODE/csvprocessing/test.csv')
# split_df[114]